In [1]:
# CS 6316 ML Project: LASSO
# Created on Wed Apr 29 22:23 2020 
# @author: sl4bq

import time
import os
import shutil
import pandas as pd
from datetime import datetime
from dateutil import tz
import warnings
import itertools

import math
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split



In [2]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import seaborn as sns

In [3]:
# Remove the NaN rows and columns

def removeNaN(curr_feat_1, curr_feat_2):
    curr_feat_1_nan = np.array([])  # numpy.float64
    curr_feat_2_nan = np.array([])  # numpy.float64
    posi = 0
    # *** Check for and remove the Nan rows in from ONLY [curr_feat_2] feature rows
    for posi in range(len(curr_feat_2)):
        if not (math.isnan(curr_feat_2[posi])):# and not(math.isnan(curr_feat_1[posi]))):
            curr_feat_1_nan = np.append(curr_feat_1_nan, curr_feat_1[posi])
            curr_feat_2_nan = np.append(curr_feat_2_nan, curr_feat_2[posi])
            if (posi < 10):
                print(curr_feat_2_nan[posi])
        if (math.isnan(curr_feat_1[posi])):
            # curr_feat_1_nan = np.append(curr_feat_1_nan, curr_feat_1[posi])
            # curr_feat_2_nan = np.append(curr_feat_2_nan, curr_feat_2[posi])
            if (posi < 10):
                print(curr_feat_2_nan[posi])
    return curr_feat_1_nan, curr_feat_2_nan

#### Import .csv dataset into pandas
print('Importing Data Now')

#filename = '/Users/Mandala/PycharmProjects/ML_Project/windowing_results.csv'
filename = 'combined_rec.csv'

# Read data from csv file 
df_feat = pd.read_csv(filename)
df_keys = df_feat.keys()



Importing Data Now


In [4]:
df_feat

,Unnamed: 0,Unnamed: 0.1,Subject,start_time,end_time,acc_mean,acc_std,acc_50%,acc_eng,acc_var,...,gyro_eng,gyro_var,gyro_skew,gyro_kurt,gsr_mean,gsr_median,gsr_variance,gsr_skewness,gsr_kurtosis,Activity
0,0,0,1,2019-11-21 09:35:27,2019-11-21 09:35:29,1.488039,0.008815,1.487741,5028.758152,0.000078,...,2444.644866,0.203803,0.492299,0.014721,1.355378,1.332223,0.000256,0.699293,-0.627312,"(0,0)"
1,1,1,1,2019-11-21 09:35:28,2019-11-21 09:35:30,1.487708,0.008177,1.487741,5061.915525,0.000067,...,2203.410455,0.208016,0.772472,0.362000,1.339162,1.318903,0.000125,-0.100332,-1.238669,"(0,0)"
2,2,2,1,2019-11-21 09:35:29,2019-11-21 09:35:31,1.484074,0.007623,1.482889,5039.399575,0.000058,...,2120.945536,0.247652,1.072809,0.951265,1.326166,1.306471,0.000135,0.010484,-1.243451,"(0,0)"
3,3,3,1,2019-11-21 09:35:30,2019-11-21 09:35:32,1.487552,0.007213,1.488377,5076.308247,0.000052,...,2354.323033,0.314183,1.283556,1.183364,1.313587,1.294483,0.000113,0.140383,-1.149240,"(0,0)"
4,4,4,1,2019-11-21 09:35:31,2019-11-21 09:35:33,1.486918,0.006790,1.488377,5047.650638,0.000046,...,3478.688197,0.451992,1.079871,0.786448,1.302077,1.286491,0.000085,0.139074,-1.145452,"(0,0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10286,10286,10286,6,2019-11-21 11:00:35,2019-11-21 11:00:37,1.601729,0.088580,1.566651,5370.640005,0.007846,...,25833.234557,3.165173,1.786256,4.765928,1.873922,1.857143,0.000018,1.241958,5.575090,"(2,3)"
10287,10287,10287,6,2019-11-21 11:00:36,2019-11-21 11:00:38,1.584692,0.055840,1.579270,5725.213374,0.003118,...,30604.610546,2.992684,1.558248,4.272371,1.872439,1.838095,0.000022,0.428993,5.263945,"(2,3)"
10288,10288,10288,6,2019-11-21 11:00:37,2019-11-21 11:00:39,1.634041,0.070983,1.651781,6096.616139,0.005039,...,33008.076912,2.628023,1.724908,5.127654,1.869794,1.838095,0.000021,0.490440,4.582487,"(2,3)"
10289,10289,10289,6,2019-11-21 11:00:38,2019-11-21 11:00:40,1.701175,0.045277,1.706349,6591.398275,0.002050,...,29859.735732,2.380862,1.807932,6.491142,1.866879,1.838095,0.000020,0.256191,3.974350,"(2,3)"


In [5]:
print(df_keys)
len(df_keys)

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Subject', 'start_time', 'end_time',
       'acc_mean', 'acc_std', 'acc_50%', 'acc_eng', 'acc_var', 'acc_skew',
       'acc_kurt', 'gyro_mean', 'gyro_std', 'gyro_50%', 'gyro_eng', 'gyro_var',
       'gyro_skew', 'gyro_kurt', 'gsr_mean', 'gsr_median', 'gsr_variance',
       'gsr_skewness', 'gsr_kurtosis', 'Activity'],
      dtype='object')


25

In [12]:
# Data Preprocessing
df = df_feat.copy()
df = df.dropna()

Xs = df[df_keys[5:24]]
y = df['Activity']

print(Xs)
print(y)

       acc_mean   acc_std   acc_50%      acc_eng   acc_var  acc_skew  \
0      1.488039  0.008815  1.487741  5028.758152  0.000078  0.084834   
1      1.487708  0.008177  1.487741  5061.915525  0.000067 -0.074702   
2      1.484074  0.007623  1.482889  5039.399575  0.000058  0.406782   
3      1.487552  0.007213  1.488377  5076.308247  0.000052 -0.003496   
4      1.486918  0.006790  1.488377  5047.650638  0.000046 -0.031493   
...         ...       ...       ...          ...       ...       ...   
10286  1.601729  0.088580  1.566651  5370.640005  0.007846  0.901604   
10287  1.584692  0.055840  1.579270  5725.213374  0.003118  0.364050   
10288  1.634041  0.070983  1.651781  6096.616139  0.005039 -0.378410   
10289  1.701175  0.045277  1.706349  6591.398275  0.002050 -0.495898   
10290  1.727280  0.024522  1.721291  4529.858341  0.000601  0.370031   

       acc_kurt  gyro_mean  gyro_std  gyro_50%      gyro_eng  gyro_var  \
0     -0.531150   0.934210  0.451446  0.892834   2444.644866 

In [13]:
# This is the data split

X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size=0.2, random_state=10)

# Training the model
model = GaussianNB()
model.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [14]:
y_pred = model.predict(X_test)
print(y_pred)

['(0,3)' '(0,1)' '(0,3)' ... '(0,3)' '(0,3)' '(0,3)']


In [15]:
len(y_pred)

2059

In [16]:
len(y_test)

2059

In [17]:
accuracy = accuracy_score(y_test, y_pred)*100
print(accuracy)

print("Number of mislabeled points out of a total {} points : {}, performance {:05.2f}%"
      .format(X_test.shape[0],(y_test != y_pred).sum(),100*(1-(y_test != y_pred).sum()/X_test.shape[0])))

10.976202039825157
Number of mislabeled points out of a total 2059 points : 1833, performance 10.98%


In [18]:


predicted = model.predict(X_test)

print(np.mean(predicted == y_test))

0.10976202039825157


In [19]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, predicted))

[[  0  14   0  35   0   0   2   3   0   0   0   0]
 [  0  53   0 151   0   0   5   6   0   1   2   0]
 [  0  76   0 136   0   0   2   4   0   3   1   1]
 [  0  39   1 130   0   0   2   5   0   0   0   1]
 [  0  10   0  30   0   0   1   0   0   1   2   0]
 [  0  54   4 124   0   0   2   2   0   4   1   0]
 [  0  79   6 150   0   0  24  23   0  14  11   1]
 [  0  33   1 143   0   0   1   6   0   1   1   1]
 [  0  14   1  20   0   0   4   2   0   3   0   0]
 [  0  56   2  84   0   0   3   7   0   1   1   0]
 [  0  59   2 161   0   0  24  15   0  10  12   2]
 [  0  38   3 104   0   0   8   2   0  16   2   0]]


In [20]:
df_video = df.loc[(df.Activity == "(0,0)") | (df.Activity == "(0,1)") | (df.Activity == "(0,2)") | (df.Activity == "(0,3)")]
df_noeval = df.loc[(df.Activity == "(1,0)") | (df.Activity == "(1,1)") | (df.Activity == "(1,2)") | (df.Activity == "(1,3)")]
df_eval = df.loc[(df.Activity == "(2,0)") | (df.Activity == "(2,1)") | (df.Activity == "(2,2)") | (df.Activity == "(2,3)")]

In [21]:
df_video

,Unnamed: 0,Unnamed: 0.1,Subject,start_time,end_time,acc_mean,acc_std,acc_50%,acc_eng,acc_var,...,gyro_eng,gyro_var,gyro_skew,gyro_kurt,gsr_mean,gsr_median,gsr_variance,gsr_skewness,gsr_kurtosis,Activity
0,0,0,1,2019-11-21 09:35:27,2019-11-21 09:35:29,1.488039,0.008815,1.487741,5028.758152,0.000078,...,2444.644866,0.203803,0.492299,0.014721,1.355378,1.332223,0.000256,0.699293,-0.627312,"(0,0)"
1,1,1,1,2019-11-21 09:35:28,2019-11-21 09:35:30,1.487708,0.008177,1.487741,5061.915525,0.000067,...,2203.410455,0.208016,0.772472,0.362000,1.339162,1.318903,0.000125,-0.100332,-1.238669,"(0,0)"
2,2,2,1,2019-11-21 09:35:29,2019-11-21 09:35:31,1.484074,0.007623,1.482889,5039.399575,0.000058,...,2120.945536,0.247652,1.072809,0.951265,1.326166,1.306471,0.000135,0.010484,-1.243451,"(0,0)"
3,3,3,1,2019-11-21 09:35:30,2019-11-21 09:35:32,1.487552,0.007213,1.488377,5076.308247,0.000052,...,2354.323033,0.314183,1.283556,1.183364,1.313587,1.294483,0.000113,0.140383,-1.149240,"(0,0)"
4,4,4,1,2019-11-21 09:35:31,2019-11-21 09:35:33,1.486918,0.006790,1.488377,5047.650638,0.000046,...,3478.688197,0.451992,1.079871,0.786448,1.302077,1.286491,0.000085,0.139074,-1.145452,"(0,0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8617,8617,8617,6,2019-11-21 10:07:33,2019-11-21 10:07:35,1.619621,0.229257,1.583901,5782.202859,0.052559,...,30935.102936,3.214207,1.517323,3.267856,1.500604,1.482051,0.000018,0.787991,3.596571,"(0,3)"
8618,8618,8618,6,2019-11-21 10:07:34,2019-11-21 10:07:36,1.745339,0.253267,1.761842,7029.333885,0.064144,...,17068.952182,0.948083,0.243240,-0.263649,1.497648,1.482051,0.000015,0.904137,5.416618,"(0,3)"
8619,8619,8619,6,2019-11-21 10:07:35,2019-11-21 10:07:37,1.978003,0.174013,2.030661,8910.642958,0.030280,...,14010.378393,0.707219,0.050833,-0.577979,1.495637,1.480586,0.000012,1.239530,7.910456,"(0,3)"
8620,8620,8620,6,2019-11-21 10:07:36,2019-11-21 10:07:38,1.981775,0.177644,2.030661,8955.207112,0.031557,...,11658.413630,0.591782,0.046189,-0.601635,1.495160,1.480586,0.000010,1.316096,7.396589,"(0,3)"


In [22]:
def NBC(x):
    y = x.copy()
    df = y.dropna()

    Xs = df[df_keys[5:24]]
    y = df['Activity']
    
    X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size=0.2, random_state=10)
    
    model = GaussianNB()
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)*100
    print(accuracy)

In [23]:
NBC(df_video)

28.634361233480178


In [24]:
NBC(df_noeval)

34.285714285714285


In [25]:
NBC(df_eval)

31.366459627329192
